In [3]:
import cv2
import os
import pandas as pd
from deepface import DeepFace
import joblib
import face_recognition
import time

class RealTimeFaceRecognizer:
    def __init__(self, student_model_path, teacher_model_path):
        self.student_model = joblib.load(student_model_path)
        self.teacher_model = joblib.load(teacher_model_path)

    def recognize_faces_and_emotions(self, video_capture, total_periods, period_duration):
        # Create a directory to store period Excel files
        os.makedirs('period_excel_files', exist_ok=True)

        # Load teacher data from Excel file
        teacher_data = pd.read_excel('teachers.xlsx')

        # Dictionary to store the mode emotion for each student
        mode_emotions = {}
        students = []

        for period_number in range(1, total_periods + 1):
            student_emotions = {}

            # Run detect_teacher for 10 seconds
            teacher_name, teacher_subject = self.detect_teacher(video_capture, self.teacher_model, teacher_data, 10)
            students.append(teacher_subject)

            # Loop for the duration of the period
            for _ in range(period_duration * 60):
                # Capture frame-by-frame
                ret, frame = video_capture.read()

                # Convert the frame to RGB
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Find all the faces in the frame
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                # Loop through each face found in the frame
                for face_encoding in face_encodings:
                    # Perform face recognition
                    predictions = self.student_model.predict([face_encoding])
                    name = predictions[0]

                    # Perform emotion detection
                    try:
                        emotion_results = DeepFace.analyze(rgb_frame, actions=['emotion'], enforce_detection=False)
                        emotion = emotion_results[0]['dominant_emotion']
                    except ValueError:
                        emotion = "Unknown"

                    # Add emotion to the dictionary
                    if name in student_emotions:
                        student_emotions[name].append(emotion)
                    else:
                        student_emotions[name] = [emotion]

            # Convert student emotions to a DataFrame
            emotions_df = pd.DataFrame.from_dict(student_emotions, orient='index')

            # Write DataFrame to Excel file without column names
            emotions_df.to_excel(f'period_excel_files/period_{period_number}_emotions.xlsx', header=False)

            # Calculate mode emotion for each student
            for student, emotions in student_emotions.items():
                if student in mode_emotions:
                    mode_emotions[student].append(max(set(emotions), key=emotions.count))
                else:
                    mode_emotions[student] = [max(set(emotions), key=emotions.count)]

        # Create DataFrame for mode emotions
        mode_df = pd.DataFrame.from_dict(mode_emotions, orient='index')

        # Add column names for mode emotions DataFrame with subjects
        mode_df.columns = [f'Period {i} - {students[i-1]}' for i in range(1, total_periods + 1)]
        mode_df.index.name = 'Name'

        # Write DataFrame to final Excel file
        mode_df.to_excel('emotion_student.xlsx')

    def detect_teacher(self, video_capture, model, teacher_data, total_time):
        start_time = time.time()
        while True:
            # Check if total_time has passed
            if time.time() - start_time > total_time:
                print("No teacher detected within the specified time.")
                return None, None

            # Capture frame-by-frame
            ret, frame = video_capture.read()

            # Convert the frame from BGR color to RGB color (required for face_recognition library)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Find all the faces and face encodings in the current frame
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            # Loop through each face found in the frame
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Perform face recognition
                predictions = model.predict([face_encoding])
                name = predictions[0]

                # Check if the recognized face exists in teacher data
                teacher_info = teacher_data[teacher_data['Name'] == name]
                if not teacher_info.empty:
                    teacher_name = teacher_info['Name'].values[0]
                    teacher_subject = teacher_info['Subject'].values[0]
                    return teacher_name, teacher_subject

                # Draw a rectangle around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                # Draw a label with the name above the face
                cv2.rectangle(frame, (left, top - 35), (right, top), (0, 255, 0), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (255, 255, 255), 1)

            # Display the resulting frame
            cv2.imshow('Video', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # If no face is detected, increase the scan time by 5 seconds
            if len(face_locations) == 0 and time.time() - start_time >= total_time:
                print("No face detected. Extending scanning time by 5 seconds.")
                start_time = time.time()
                total_time += 5

if __name__ == "__main__":
    student_model_path = "students_trained_model.pkl"  # Path to your student trained model
    teacher_model_path = "trained_model.pkl"  # Path to your teacher trained model

    # Initialize the camera
    video_capture = cv2.VideoCapture(0)  # 0 for default camera, change it according to your camera setup

    # Input total number of periods and period duration
    total_periods = int(input("Enter total number of periods: "))
    period_duration = int(input("Enter period duration in minutes: "))

    # Perform real-time face recognition with emotion detection for each period
    recognizer = RealTimeFaceRecognizer(student_model_path, teacher_model_path)
    recognizer.recognize_faces_and_emotions(video_capture, total_periods, period_duration)


Enter total number of periods: 3
Enter period duration in minutes: 1
